In [ ]:
import json
import pandas as pd
import requests
import time

In [ ]:
# Fetch tickers from the openfigi api for a list of ISINs
def fetch_tickers_from_openfigi(isins: list, exchcode: str = 'US') -> dict:
    """ 
    Fetch tickers from the openfigi api for a list of ISINs
    :param isins: list of ISINs
    :param exchcode: exchange code
    :return: dictionary with ISINs as keys and tickers as values
    """

    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    openfigi_headers = {'Content-Type': 'application/json'}

    # Prepare request data for multiple ISINs
    openfigi_data = []
    for isin in isins:
        if isin == '-':
            continue
        openfigi_data.append({"idType": "ID_ISIN", "idValue": isin, "exchCode": exchcode})

    # Create a dictionary to store the tickers
    tickers_dict = {}
    # Process each ISIN
    for idx, data in enumerate(openfigi_data):
        # Send request
        openfigi_response = requests.post(openfigi_url, headers=openfigi_headers, json=[data])

        # Check if request was successful
        if openfigi_response.status_code == 200:
            try:
                openfigi_response_json = openfigi_response.json()
                if 'data' in openfigi_response_json[0] and 'ticker' in openfigi_response_json[0]['data'][0]:
                    tickers_dict[isins[idx]] = openfigi_response_json[0]['data'][0]['ticker']
                else:
                    tickers_dict[isins[idx]] = None
            except json.JSONDecodeError:
                print("Error decoding JSON for ISIN:", isins[idx])
                tickers_dict[isins[idx]] = None
        else:
            print("Request failed for ISIN:", isins[idx], "with status code:", openfigi_response.status_code)
            tickers_dict[isins[idx]] = None

        time.sleep(3)  # api limit is 25 request per minute

    with open('tickers_response.json', 'w') as file:
        json.dump(tickers_dict, file)

    return tickers_dict

In [8]:
companies_df = pd.read_excel('data/ishares-msci-world-etf-holdings-29.02.2024.xlsx')
fetch_tickers_from_openfigi(companies_df['ISIN'].tolist())

Request failed for ISIN: US00287Y1091 with status code: 504
Request failed for ISIN: US2546871060 with status code: 504
Request failed for ISIN: US38141G1040 with status code: 504
Request failed for ISIN: US92532F1003 with status code: 503


{'US5949181045': 'MSFT',
 'US0378331005': 'AAPL',
 'US67066G1040': 'NVDA',
 'US0231351067': 'AMZN',
 'US30303M1027': 'META',
 'US02079K3059': 'GOOGL',
 'US02079K1079': 'GOOG',
 'US5324571083': 'LLY',
 'US11135F1012': 'AVGO',
 'US88160R1014': 'TSLA',
 'US46625H1005': 'JPM',
 'US0846707026': 'BRK/B',
 'US91324P1021': 'UNH',
 'US92826C8394': 'V',
 'US30231G1022': 'XOM',
 'US57636Q1040': 'MA',
 'US4781601046': 'JNJ',
 'DK0062498333': 'NONOF',
 'NL0010273215': 'ASMLF',
 'US4370761029': 'HD',
 'US7427181091': 'PG',
 'US22160K1051': 'COST',
 'US58933Y1055': 'MRK',
 'US0079031078': 'AMD',
 'US00287Y1091': None,
 'US79466L3024': 'CRM',
 'CH0038863350': 'NSRGF',
 'US1667641005': 'CVX',
 'US64110L1061': 'NFLX',
 'US9311421039': 'WMT',
 'JP3633400001': 'TOYOF',
 'US00724F1012': 'ADBE',
 'FR0000121014': 'LVMHF',
 'US1912161007': 'KO',
 'US0605051046': 'BAC',
 'IE00B4BNMY34': 'ACN',
 'US7134481081': 'PEP',
 'US8835561023': 'TMO',
 'IE000S9YS762': 'LIN',
 'US5801351017': 'MCD',
 'CH0012005267': 'NVSE

just to safe the failed request:
```
Request failed for ISIN: US00287Y1091 with status code: 504
Request failed for ISIN: US2546871060 with status code: 504
Request failed for ISIN: US38141G1040 with status code: 504
Request failed for ISIN: US92532F1003 with status code: 503
```

In [9]:
companies_df = pd.read_excel('data/ishares-core-sp-500-etf-29.02.2024.xlsx')
fetch_tickers_from_openfigi(companies_df['ISIN'].tolist())

{'US5949181045': 'MSFT',
 'US0378331005': 'AAPL',
 'US67066G1040': 'NVDA',
 'US0231351067': 'AMZN',
 'US30303M1027': 'META',
 'US02079K3059': 'GOOGL',
 'US0846707026': 'BRK/B',
 'US02079K1079': 'GOOG',
 'US5324571083': 'LLY',
 'US11135F1012': 'AVGO',
 'US88160R1014': 'TSLA',
 'US46625H1005': 'JPM',
 'US91324P1021': 'UNH',
 'US92826C8394': 'V',
 'US30231G1022': 'XOM',
 'US57636Q1040': 'MA',
 'US4781601046': 'JNJ',
 'US4370761029': 'HD',
 'US7427181091': 'PG',
 'US22160K1051': 'COST',
 'US58933Y1055': 'MRK',
 'US0079031078': 'AMD',
 'US00287Y1091': 'ABBV',
 'US79466L3024': 'CRM',
 'US1667641005': 'CVX',
 'US64110L1061': 'NFLX',
 'US00724F1012': 'ADBE',
 'US9311421039': 'WMT',
 'US0605051046': 'BAC',
 'IE00B4BNMY34': 'ACN',
 'US1912161007': 'KO',
 'US7134481081': 'PEP',
 'US8835561023': 'TMO',
 'IE000S9YS762': 'LIN',
 'US5801351017': 'MCD',
 'US0028241000': 'ABT',
 'US2546871060': 'DIS',
 'US9497461015': 'WFC',
 'US17275R1023': 'CSCO',
 'US4612021034': 'INTU',
 'US4581401001': 'INTC',
 'U

In [16]:
def insert_new_ticker(ticker_file:str, df:pd.DataFrame, updated_file:str):
    with open(ticker_file, 'r') as file:
        json_data = json.load(file)

    df['Ticker'] = df['ISIN'].map(json_data).fillna('NaN')
    df.to_excel(updated_file, index=False)
    pass

In [17]:
insert_new_ticker('data/ishares_msci_world_ticker.json', pd.read_excel('data/ishares-msci-world-etf-holdings-29.02.2024.xlsx'), 'data/msci-world-updated.xlsx')

In [18]:
insert_new_ticker('data/ishares_sp_500_ticker.json', pd.read_excel('data/ishares-core-sp-500-etf-29.02.2024.xlsx'), 'data/sp-500-updated.xlsx')